In [1]:
class TreeNode:
  def __init__(self, label=''):
    self.label = label
    self.children = []

  def is_wffr(self, formula):
    if formula == '':
        return False
    if formula[0] == '¬':
        return self.is_wffr(formula[1:])
    if formula[0] == '(':
        if formula[-1] != ')':
            return False
        count = 1
        index = -1
        for i in range(1, len(formula) - 1):
            if formula[i] == '*' and count == 1:
                index = i
                break
            elif formula[i] == '(': 
                count += 1
            elif formula[i] == ')': 
                count -= 1
        return index > 0 and self.is_wffr(formula[1:index]) and self.is_wffr(formula[index + 1:-1])
    return not "(" in formula and not ")" in formula and not "*" in formula

  def is_wff(self, formula):
    list_op = ["∧", "V", "≡", "⊃"]
    for op in list_op: formula = formula.replace(op, "*")
    formula = formula.replace(" ", "")
    return self.is_wffr(formula)

  def getTree(self, formula):
    formula = formula.replace(" ", "")
    list_op = ["∧", "V", "≡", "⊃"]
    if formula[0] == '¬':
      node = TreeNode('¬')
      node.children.append(node.getTree(formula[1:]))
      return node
    else:
      count = 1
      index = -1
      for i in range(1,len(formula)-1):
        if formula[i] in list_op and count == 1:
          index = i
          break
        elif formula[i] == '(': count+=1
        elif formula[i] == ')': count-=1
      #print(index)
      if index>=0:
        node = TreeNode(formula[index])
        node.children.append(node.getTree(formula[1:index]))
        node.children.append(node.getTree(formula[index+1:-1]))
      else:
        node = TreeNode(formula)
        print(formula)
    return node

  def print_tree(self, level=0):
      print('  ' * level + self.label)
      for child in self.children:
          child.print_tree(level + 1)

In [2]:
# Exemplo de uso
frase = "((P ∧ R) ∧ R)"
tree = TreeNode()
print(tree.is_wff(frase))
tree = tree.getTree(frase)
tree.print_tree()

True
P
R
R
∧
  ∧
    P
    R
  R


In [91]:
class Tableaux:
    def __init__(self, root):
        self.root = root.replace(" ", "")
    
    def lista(self):
        # Retorna uma lista com as fórmulas do tableau
        # separa a string por virgula e adiciona as novas strings na lista
        lista1 = [self.root.split(",")]
        # itera sobre cada string da lista procurando por ∴ tudo que estiver antes de ∴ é adicionado a lista e tudo que estiver depois é adicionado a lista com ¬ à esquerda
        lista2 = []
        for i in lista1:
            for j in i:
                if "∴" in j:
                    lista2.append(j.split("∴")[0])
                    lista2.append("¬" + j.split("∴")[1])
                else:
                    lista2.append(j)
        return lista2
    
    # itera sobre cada string da lista e verifica se é uma fórmula atômica, se for, o item é adicionado a lista de fórmulas atômicas, se não for, o item é adicionado a lista de fórmulas compostas
    def atom_and_comp(self):
        atom = []
        comp = []
        for i in self.lista():
            # se a string for uma fórmula atômica, ou seja, não tiver ∧, ∨, ⊃ e ≡ dentro da string ela é adicionada a lista de fórmulas atômicas
            if "∧" not in i and "∨" not in i and "⊃" not in i and "≡" not in i:
                atom.append(i)
            else:
                comp.append(i)
        return atom, comp
    
# itera sobre cada string da lista de fórmulas compostas e verifica se tem V, ∧, ⊃, ≡, se tiver, a string é separada pelo operador e adicionada a lista de fórmulas atômica2
    def comp_to_atom(self):
        atom, comp = self.atom_and_comp()
        atom2 = []
        # verificar todos os casos em que a fórmula composta não irá gerar nó
        for i in comp:
            if "¬(" in i:
                i = i.replace("¬(", "").replace(")", "")
                if "¬" in i[1]:
                    atom.append(i[2])
                elif "∧" in i:
                    atom2.append("¬" + i.split("∧")[0])
                    atom2.append("¬" + i.split("∧")[1])
                elif "∨" in i:
                    atom.append("¬" + i.split("∨")[0])
                    atom.append("¬" + i.split("∨")[1])
                elif "⊃" in i:
                    atom.append(i.split("⊃")[0])
                    atom.append("¬" + i.split("⊃")[1])
                elif "≡" in i:
                    atom2.append(["¬" + i.split("≡")[0], i.split("≡")[1]])
                    atom2.append([i.split("≡")[0], "¬" + i.split("≡")[1]])
            elif "¬(" not in i:
                i = i.replace("(", "").replace(")", "")
                if "∧" in i:
                    atom.append(i.split("∧")[0])
                    atom.append(i.split("∧")[1])
                elif "∨" in i:
                    atom2.append(i.split("∨")[0])
                    atom2.append(i.split("∨")[1])
                elif "⊃" in i:
                    atom2.append("¬" + i.split("⊃")[0])
                    atom2.append(i.split("⊃")[1])
                elif "≡" in i:
                    atom2.append([i.split("≡")[0], i.split("≡")[1]])
                    atom2.append(["¬" + i.split("≡")[0], "¬" + i.split("≡")[1]])
        # iteraremos sobre a lista de fórmulas atômicas 2 e adicionaremos a uma lista principal listas contendo a lista de fórmulas atômicas + o item da lista de fórmulas atomicas 2
        if atom2 == []:
            return atom
        else:
            lista_atom = []
            for i in atom2:
                if type(i) == list:
                    lista_atom.append(atom + [i[0]] + [i[1]])
                else:
                    lista_atom.append(atom + [i])
            return lista_atom
        
    def contradictions(self):
        lista_atom = self.comp_to_atom()
        # iteraremos sobre a lista principal e verificaremos se há fórmulas contraditórias, se houver, toda a lista é apagada. Se não houver, a lista é mantida. Caso os itens da lista sejam listas devemos iterar sobre elas.
        for i in lista_atom:
            if type(i) == list:
                for j in i:
                    if j.startswith("¬") and j[1:] in i:
                        i.clear()
                    elif "¬" + j in i:
                        i.clear()
            else:
                if i.startswith("¬") and i[1:] in lista_atom:
                    lista_atom.clear()
                elif "¬" + i in lista_atom:
                    lista_atom.clear()
        return lista_atom

In [92]:
# testando a classe Tableaux
tableaux = Tableaux("(P ⊃ Q), P ∴ Q")
print(tableaux.lista())
print(tableaux.atom_and_comp())
print(tableaux.comp_to_atom())
print(tableaux.contradictions())


['(P⊃Q)', 'P', '¬Q']
(['P', '¬Q'], ['(P⊃Q)'])
[['P', '¬Q', '¬P'], ['P', '¬Q', 'Q']]
[[], []]
